In [6]:
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
ontology_and_seq__fp = "../data/intermediary/drosophila_protein_ontology_and_seqs.csv"
df = pd.read_csv(ontology_and_seq__fp)
relevant_subset = df[df.qualifier.isin(["enables", "involved_in"])].dropna()
interesting_go_names = [name for (name, freq) in relevant_subset.go_name.value_counts().to_dict().items() if 1 < freq]  # <- probably need to change the filter step !!
df = df[df.go_name.isin(interesting_go_names)]
one_row_per_gene = pd.DataFrame(index=df.seq.unique(), columns=interesting_go_names).fillna(0)
for _, row in df.iterrows():
    one_row_per_gene.loc[row.seq, row.go_name] = 1
one_row_per_gene = one_row_per_gene.reset_index().rename(columns={"index": "seq"})
one_row_per_gene.head()

,seq,asymmetric neuroblast division,nuclear-transcribed mRNA poly(A) tail shortening,establishment or maintenance of neuroblast polarity,protein import into nucleus,methionine biosynthetic process,sarcomere organization,ribosome biogenesis,mRNA export from nucleus,establishment of imaginal disc-derived wing hair orientation,...,wing disc morphogenesis,"terminal branching, open tracheal system",axoneme assembly,eggshell chorion gene amplification,sodium ion transmembrane transport,adenylate cyclase-activating G protein-coupled receptor signaling pathway,peripheral nervous system neuron development,establishment of planar polarity of embryonic epithelium,NAD biosynthetic process,axonemal dynein complex assembly
0,MDCFKVFEVVFQSEINPLLLIPAVATIALTLCCYCYHGYQWIRDRR...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MFTTRKEVDAHVHKMLGKLQPGRERDIKGLAVARLYMKVQEYPKAI...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MEPHFFFTVLWMLLMGTSSTYAQEIFGYCRTPDENSGTCINLRECG...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MSNVHQFDTQTMAESPQIRRDMGRLCATWPSKDSEDGAGTALRAAT...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MALVYGVEKKTVPTHMKFVMGGTSGMLATCIVQPLDLLKTRMQISG...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df = pd.read_csv(ontology_and_seq__fp)
relevant_subset = df[df.qualifier.isin(["enables", "involved_in"])].dropna()
# !! probably need to change the filter step !!
interesting_go_names = [
    name for (name, freq) in relevant_subset.go_name.value_counts().to_dict().items() if 25 < freq
]  
df = df[df.go_name.isin(interesting_go_names)]
df = pd.DataFrame(index=df.seq.unique(), columns=interesting_go_names).fillna(0)
for _, row in df.iterrows():
    df.loc[row.seq, row.go_name] = 1
df = one_row_per_gene.reset_index().rename(columns={"index": "seq"})
df["training"] = 0
df["training"] = df.training.apply(lambda _: random.random() < 0.75)

class SeqOntologyDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.X = df.loc[:, "seq"]
        self.y = df.loc[:, interesting_go_names]
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        return self.X[i], self.y[i, :]
    
dl_test = DataLoader(SeqOntologyDataset(df[~df.training]), batch_size=32)
dl_train = DataLoader(SeqOntologyDataset(df[df.training]), batch_size=32)